<span type="title">SSH Integration</span> | <span type="update">2018-10-26</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要介绍 Spring、Spring MVC、Hibernate 的整合。</p></span>

# Spring 和 Hibernate 的整合

在经典的 Hibernate 程序中，采用 hibernate.cfg.xml 配置文件配置数据库、数据源、域模型相关策略（显示 SQL 语句、格式化 SQL 语句，开启二级缓存、查询缓存、启用数据库方言等等）。整合 Hibernate 的目的在于，让 WEB 程序能够快速的通过 Hibernate 的 SessionFactory 获得 Session，并且进行数据交互操作。在 Hibernate 的章节，我们介绍了基于 thread 的线程绑定获取 Session 的方式。这常被直接写入 Data Access Object 中，作为类实例变量，进行数据交互使用。

因此，我们面临的唯一问题就是在何时获得 SessionFactory 对象，一般而言，我们使用 Spring IOC 容器来管理 Hibernate 的 SessionFactory。很显然的，IOC 中创建 Bean 的方式创建 SF，可选两种方案，其一，将所有原本 Hibernate 配置信息作为参数传递进去，其二，将原本 Hibernate 配置文件保留，让 SessionFactory 对象自动寻找 Hibernate 配置文件，并且进行加载。

## 配置 SessionFactory 和 DataSource

一个整合 Spring 的 Hibernate.cfg.xml 文件如下所示：

```xml
<?xml version='1.0' encoding='utf-8'?>
<!DOCTYPE hibernate-configuration PUBLIC "-//Hibernate/Hibernate Configuration DTD//EN"
        "http://www.hibernate.org/dtd/hibernate-configuration-3.0.dtd">
<hibernate-configuration>
    <session-factory>
        <property name="hibernate.dialect">org.hibernate.dialect.MySQL8Dialect</property>
        <property name="hibernate.show_sql">true</property>
        <property name="hibernate.hbm2ddl.auto">update</property>
        <property name="hibernate.current_session_context_class">
        org.springframework.orm.hibernate5.SpringSessionContext</property>
    </session-factory>
</hibernate-configuration>
```

其中 DataSource 数据源可能除了 Hibernate，还有别的用途，因此需要配置到 IOC 容器中，然后作为参数传递给 Session Factory 的 bean 对象。因此，整合后的 Hibernate 配置文件仅仅配置：方言、SQL 显示、格式化、生成数据表的策略和二级缓存策略。

一个整合了 Hibernate 的 Spring.cfg.xml 文件如下所示，注意，这里需要配置数据源、SessionFactory 以及配置交给 Spring 管理的 Hibernate 的事务。

对于数据源而言，一般提供一个 properties 文件，放置数据敏感信息，然后使用占位符来加载信息，之后向 beans 中写入信息。数据源需要注意的是，<u>忘记写数据库地址会报找不到数据库</u>错误。

对于 SessionFacotry 而言，我们需要提供数据源 bean 引用，配置文件路径，映射模型文件路径，除了使用配置文件以外，还可以直接在 hibernateProperties 属性中提供 props 的标签，写入各个原本 Hibernate 配置的字段，作为 key - value 传入，如果同时加载了配置，则以此处为准。<u>配置此处需要注意，路径需要区分 classpath 和物理目录。对于 maven 工程，一般放置在 resource 目录下，也就是作为 classpath 路径下的目录。当进行 web 整合的时候，如果放置在此文件夹下的目录不添加 classpath 前缀，则容易发生找不到文件的异常。</u>

对于数据库方言，需要注意，如果默认使用了 MySQL5Dialect，则使用的引擎 MyISDM 不支持事务。

```xml
<?xml version="1.0" encoding="UTF-8"?>
<beans xmlns="http://www.springframework.org/schema/beans"
       xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:beans="http://www.springframework.org/schema/beans"
       xmlns:context="http://www.springframework.org/schema/context" xmlns:p="http://www.springframework.org/schema/p"
       xmlns:tx="http://www.springframework.org/schema/tx" xmlns:aop="http://www.springframework.org/schema/aop"
       xsi:schemaLocation="http://www.springframework.org/schema/beans
       http://www.springframework.org/schema/beans/spring-beans.xsd
       http://www.springframework.org/schema/context http://www.springframework.org/schema/context/spring-context.xsd
       http://www.springframework.org/schema/tx http://www.springframework.org/schema/tx/spring-tx.xsd
       http://www.springframework.org/schema/aop http://www.springframework.org/schema/aop/spring-aop.xsd
">
    <!--在此处导入 DataSource 而不是 Hibernate 中, 使用 Java Class 而不是 C3P0.xml 配置-->
    <context:property-placeholder location="classpath:database.properties" />
    <bean class="com.mchange.v2.c3p0.ComboPooledDataSource" id="dataSource"
          p:user="${jdbc.user}" p:password="${jdbc.password}" p:driverClass="${jdbc.driverClass}"
          p:initialPoolSize="${jdbc.initPoolSize}" p:maxPoolSize="${jdbc.maxPoolSize}"
          p:jdbcUrl="${jdbc.jdbcUrl}"
    />
    <!--在此处配置 Hibernate 的 SessionFactory， 同时提供其配置文件和映射文件-->
    <bean class="org.springframework.orm.hibernate5.LocalSessionFactoryBean" id="sessionFactory"
          p:dataSource-ref="dataSource"
          p:configLocation="classpath:hibernate.cfg.xml"
          p:mappingLocations="classpath:hbm/*.hbm.xml">
        <property name="hibernateProperties">
            <props>
                <prop key="hibernate.dialect">org.hibernate.dialect.MySQL8Dialect</prop>
                <prop key="hibernate.show_sql">true</prop>
                <prop key="hibernate.format_sql">false</prop>
                <prop key="hibernate.hbm2ddl.auto">update</prop>
            </props>
        </property>
    </bean>
    <!--配置 Spring 声明式事务：事务管理器、事务属性、事务切点-->
    <bean class="org.springframework.orm.hibernate5.HibernateTransactionManager" id="transactionManager"
          p:sessionFactory-ref="sessionFactory"/>
    <tx:advice id="txAdvice">
        <tx:attributes>
            <tx:method name="get*" read-only="true"/>
            <tx:method name="purchase" propagation="REQUIRES_NEW"/>
            <tx:method name="*" />
        </tx:attributes>
    </tx:advice>
    <aop:config>
        <aop:pointcut id="txPointcut" expression="execution(* com.mazhangjing.learn.temp.service.*.*(..))"/>
        <aop:advisor advice-ref="txAdvice" pointcut-ref="txPointcut" />
    </aop:config>

    <context:component-scan base-package="com.mazhangjing.learn.temp">
        <context:exclude-filter type="annotation" expression="org.springframework.stereotype.Controller" />
        <context:exclude-filter type="annotation" expression="org.springframework.web.bind.annotation.ControllerAdvice" />
    </context:component-scan>
</beans>
```

## 启用声明式事务支持

除了配置数据源、提供数据源、映射文件、配置文件以配置 SessionFactory，还需要配置声明式事务支持。因为主要采用了 XML 方式进行配置，因此，这里直接使用手动的方式，而不是基于注解，来配置事务。

事务需要提供事务管理器，此管理器实现了对于事务管理在 Spring 的实现，此外，需要提供 tx:advice 标签来定义此管理器的策略组。在此组中，需要配置 tx:method 来指定方法的事务的策略，常见的 get*, 一般选择只读来优化查询性能。对于特殊的服务层方法，则直接定义 tx:method 方法，比如指定事务合并级别：开新事务，而不是默认的合并入他人的事务中。此外，使用星号对于其余的事务进行默认的配置。

当我们有了事务管理器，并且配置好事务策略后，需要通过 aop:config 标签定义事务切入的位点：一般需要作用于 service 层，使用 `* xxx.service.*.*(..)` 来作用于 service 包下所有类的所有方法。使用 aop:pointcut 来定义切点，使用 aop:advisor 来应用事务策略组。

## 谨慎定义自动注解扫描

一般而言，我们直接对所有的